## PART A


In [ ]:
import tensorflow as tf
import json
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load the data from the file and preprocess it
data = []
for line in open('train.json', 'r'):
    data.append(json.loads(line))

In [35]:
# Define the class of relations to classify
relations_to_classify = ["CEO",'SUBSIDIARY_OF','DATE_OF_DEATH','RESIDENCE', 'BIRTH_PLACE', 'NATIONALITY', 'EMPLOYEE_OR_MEMBER_OF', 'EDUCATED_AT', 'POLITICAL_AFF', "CHILD_OF", 'SPOUSE', "DATE_FOUNDED", 'HEADQUARTERS', 'FOUNDED_BY']

def preprocess_text(text):
    """
    This function performs basic text preprocessing by removing punctuations, 
    converting text to lowercase, and removing extra white spaces.
    """
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub('\s+', ' ', text)
    text = text.lower().strip()
    return text

# Modify preprocess_data to return a list of relations for each passage
def preprocess_data(data):
    """
    This function preprocesses the Knowledgent dataset by extracting the 
    document text and the annotated properties for each passage. It also 
    performs text preprocessing on the document text.
    """
    X = []
    y = []
    for d in data:
        document_text = d['documentText']
        document_text = preprocess_text(document_text)
        for passage in d['passages']:
            passage_text = passage['passageText']
            passage_text = preprocess_text(passage_text)
            annotated_properties = []
            for fact in passage['exhaustivelyAnnotatedProperties']:
                annotated_properties.append(fact['propertyName'])
            # Filter for only the desired relations
            if any(prop in relations_to_classify for prop in annotated_properties):
                X.append(passage_text)
                # Convert the list of annotated properties to binary labels
                y.append([prop for prop in annotated_properties if prop in relations_to_classify])
    return X, y


In [ ]:

# Modify the prediction function to output the relation(s) instead of binary labels
def predict_relation(model, text):
    """
    This function takes a trained model and an input text, preprocesses the text,
    and returns the predicted relation(s) for the input text.
    """
    text = preprocess_text(text)
    text_bow = vectorizer.transform([text]).toarray()
    prediction = model.predict(text_bow)[0]
    # Convert the prediction to binary labels
    binary_prediction = [1 if p > 0.5 else 0 for p in prediction]
    # Use relations_to_classify to determine the predicted relation(s)
    predicted_relations = [relations_to_classify[i] for i, label in enumerate(binary_prediction) if label == 1]
    return predicted_relations


X, y = preprocess_data(data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract bag-of-words features from the text data
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)
import numpy as np



# Define the model architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_bow.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(14, activation='sigmoid') # modified last layer
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
# Train the model
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the lists of lists to binary labels
mlb = MultiLabelBinarizer(classes=relations_to_classify)
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)

# Train the model
history = model.fit(X_train_bow.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test_bow.toarray(), y_test))

Epoch 1/10
108/108 [==============================] - 16s 133ms/step - loss: 0.3217 - accuracy: 0.1365 - val_loss: 0.2241 - val_accuracy: 0.5574
Epoch 2/10
108/108 [==============================] - 15s 138ms/step - loss: 0.1947 - accuracy: 0.4775 - val_loss: 0.1513 - val_accuracy: 0.6431
Epoch 3/10
108/108 [==============================] - 14s 132ms/step - loss: 0.1115 - accuracy: 0.7566 - val_loss: 0.1313 - val_accuracy: 0.6941
Epoch 4/10
108/108 [==============================] - 16s 148ms/step - loss: 0.0647 - accuracy: 0.8731 - val_loss: 0.1326 - val_accuracy: 0.6941
Epoch 5/10
108/108 [==============================] - 16s 148ms/step - loss: 0.0475 - accuracy: 0.9096 - val_loss: 0.1328 - val_accuracy: 0.7034
Epoch 6/10
108/108 [==============================] - 15s 138ms/step - loss: 0.0355 - accuracy: 0.9325 - val_loss: 0.1411 - val_accuracy: 0.7057
Epoch 7/10
 86/108 [======================>.......] - ETA: 2s - loss: 0.0296 - accuracy: 0.9390

PART B

In [ ]:
def preprocess_data(data):
    """
    This function preprocesses the Knowledgent dataset by extracting the 
    document text and the annotated properties for each passage. It also 
    performs text preprocessing on the document text. It only appends passages
    that contain the desired relations.
    """
    X = []
    y = []
    for d in data:
        document_text = d['documentText']
        document_text = preprocess_text(document_text)
        for passage in d['passages']:
            passage_text = passage['passageText']
            passage_text = preprocess_text(passage_text)
            annotated_properties = []
            for fact in passage['exhaustivelyAnnotatedProperties']:
                annotated_properties.append(fact['propertyName'])
            # Filter for only the desired relations
            if any(prop in ["DATE_OF_BIRTH", "PLACE_OF_RESIDENCE", "BIRTHPLACE", "NATIONALITY", "EMPLOYEE_OR_MEMBER_OF", "EDUCATED_AT"] for prop in annotated_properties):
                X.append(d)
                # Convert the list of annotated properties to binary labels
                y.append([prop for prop in annotated_properties if prop in relations_to_classify])
    return X, y


In [ ]:
p,r=preprocess_data(data)
p

['when she founded apex legal document preparation services she became a premium member of the faldp florida association of legal document preparers and educated herself on the different types of divorce been offered across the state of florida and learned from the best in the industry of legal document preparation services',
 'tata chemicals europe formerly brunner mond uk limited is a ukbased chemicals company that is a subsidiary of tata chemicals limited itself a part of the indiabased tata group',
 'after five successful albums and extensive touring they disbanded after lead vocalist sandman died of a heart attack onstage in palestrina italy on july 3 1999',
 'he is a member of phi beta kappa and was a fulbright fellow in law in brazil 19801981',
 'he sheikh abdullah bin mohammed bin saud al thani arabic عبد الله بن محمد بن سعود آل ثاني is a qatari businessman politician and a member of the royal family',
 'in december 2014 he became the ceo of the qatar investment authority',
 'g

# Used to find information about nodes and relationships to create KG in neo4j

In [ ]:
desired_relations = ["12",'11','10','15','3','9']
extracted_facts = []

for data_point in data:
    for passage in data_point["passages"]:
        for fact in passage["facts"]:
            if fact["propertyId"] in desired_relations:
                extracted_facts.append(fact)
                

human_readable_list = []
for item in extracted_facts:
    human_readable_list.append(item['humanReadable'])
    human_readable_list.append('para_end')



In [ ]:
human_readable_list

['<بد الله بن محمد بن سعود آل ثاني> <NATIONALITY> <Qatari>',
 'para_end',
 '<Abdullah bin Mohammed bin Saud Al Thani> <NATIONALITY> <Qatari>',
 'para_end',
 '<Jim Harris> <NATIONALITY> <American>',
 'para_end',
 '<James Patrick Harris> <NATIONALITY> <American>',
 'para_end',
 '<I> <EDUCATED_AT> <Specs Howard School of Media Arts>',
 'para_end',
 '<I> <EDUCATED_AT> <where>',
 'para_end',
 '<my> <EDUCATED_AT> <where>',
 'para_end',
 '<my> <EDUCATED_AT> <Specs Howard School of Media Arts>',
 'para_end',
 '<I> <EDUCATED_AT> <Specs Howard School of Media Arts>',
 'para_end',
 '<I> <EDUCATED_AT> <Specs Howard School of Media Arts>',
 'para_end',
 '<I> <EDUCATED_AT> <where>',
 'para_end',
 '<myself> <EDUCATED_AT> <where>',
 'para_end',
 '<I> <EDUCATED_AT> <Specs Howard School of Media Arts>',
 'para_end',
 '<myself> <EDUCATED_AT> <Specs Howard School of Media Arts>',
 'para_end',
 '<I> <EDUCATED_AT> <where>',
 'para_end',
 '<I> <EDUCATED_AT> <where>',
 'para_end',
 '<my> <EDUCATED_AT> <Specs 